## Exporting models for use on client-side apps

## Preprocessing models

My model was using transfer learning from Resnet-50. So, I will need to either
- create a model with both, or
- extract both models separately and use the predictions of one in the other

Architecturally, it makes more sense to go the later way, because then the last layer can be easily replaced while using this for other apps

Note: Use the `dog-linux.yml` tog enerate the conda environment to run this model

#### Loading the saved weights for the models

In [1]:
from keras.models import load_model

base_resnet = load_model('./keras_models/base_resnet.hdf5')
last_layers_model = load_model('./keras_models/weights.extra_layers.hdf5')
last_layers_model.summary()

## ignore the warning for base_resnet!

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 500)               1024500   
_________________________________________________________________
dropout_7 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 133)               66633     
Total params: 1,091,133.0
Trainable params: 1,091,133.0
Non-trainable params: 0.0
_________________________________________________________________


#### Testing the workflow

In [ ]:
import numpy as np
dog_names = np.load('./labels/dog_names.npy')
print('Number of dogs:', len(dog_names))

In [ ]:
### function converts an image path into a tensor that can be input into the resnet model
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

In [ ]:
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing import image    
import numpy as np

def predict_breed(img_path):
    tensor = path_to_tensor(img_path)
    bottleneck_feature = base_resnet.predict(preprocess_input(tensor))
    predicted_vector = last_layers_model.predict(bottleneck_feature)
    return dog_names[np.argmax(predicted_vector)]

In [ ]:
predict_breed('./dogImages/Alaskan_malamute')